In [ ]:
!pip install --upgrade transformers==4.17
!pip install --upgrade datasets evaluate huggingface_hub torch

In [ ]:
from datasets import load_dataset
# dataset=load_dataset("yelp_review_full")

dataset = load_dataset('csv', data_files='data.csv')
dataset = dataset['train'].train_test_split(test_size=0.1)


In [ ]:
# dataset["train"][1]
print(dataset['train'][0])
print(dataset['test'][0])

{'company': 'Costco', 'ticker': 'COST', 'title': 'Walmart Connect запускает собственную систему DSP', 'text': 'Walmart Connect объявила о запуске собственной платформы для управления цифровой рекламой DSP. Об этом сообщается на сайте компании. Платформа создана в партнерстве с ведущим разработчиком в этой сфере The Trade Desk. Она будет запущена к концу октября текущего года. Благодаря омниканальной аналитике Walmart и технологиям The Trade Desk компании рассчитывают создать платформу, которая позволит клиентам эффективно расходовать средства на рекламу. Больше новостей об инвестициях вы найдете в нашем телеграм-канале «Сам ты инвестор!»', 'link': 'https://www.rbc.ru/quote/news/short_article/612651ab9a7947492b778bd7', 'sentiment': 2}
{'company': 'Alibaba', 'ticker': 'BABA', 'title': 'Ozon объединит два своих банка до конца года', 'text': 'Онлайн-ретейлер Ozon принял решение реорганизовать банковский бизнес и присоединить новый Еком Банк к купленному в 2021 году Ozon Банку. По предварит

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/23652 [00:00<?, ? examples/s]

Map:   0%|          | 0/2628 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text", "company", "ticker", "link", "title"])
tokenized_datasets = tokenized_datasets.rename_column("sentiment", "labels")
tokenized_datasets.set_format("torch")
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

print(small_train_dataset[0])
print(small_train_dataset.column_names)

{'labels': tensor(3), 'input_ids': tensor([    2,   287, 20879,  1026,  4823,  5206,  1091, 10784,  9558,   105,
          310,  8700,  9995, 18092, 15379,   117,    12,   310,  7762,  3301,
           13,  1129,   887,  9573,   971,   314,  8232, 18807,  8203,  1211,
          327, 12875, 21121,  1172,  4495,  3277, 10113,   312, 14351,     1,
        11735,    16,  1129,  3200,  3789,   794,  4495,  3277, 10113, 24047,
            1,    16,  1421,   650,   312, 28579,    16,  1046, 28896,  5548,
        21766, 19224,  3789, 13248,  1172, 15332, 11806,  8264,  3238,    12,
            1,    16,  1421,     1,    20,    16,    26,   650,   312, 28579,
           13,    16,  1154, 10658, 28847,   613,   314,  7197,    18,   282,
         6156,  7437,  1451,   328,  1904,   721,  8754, 17465, 23378,  3782,
        16777,  7197,   548,  6024,  6329,  1619, 17045,   811,  2755,  3825,
            1, 11735,    16, 16667,   938,   314,  1154, 10658, 21901,  7197,
           18, 11847, 21612, 

In [ ]:
import torch
from torch.utils.data import DataLoader
traindataloader=DataLoader(small_train_dataset,batch_size=8,shuffle=True)
testdataloader=DataLoader(small_eval_dataset,batch_size=8)

In [ ]:
from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny", num_labels=5)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29564, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, e

In [ ]:
from torch.optim import AdamW,SGD
from transformers import get_scheduler
optimizer=SGD(model.parameters(),lr=5e-5)
num_epochs=35
num_training_steps = num_epochs * len(traindataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in traindataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/103495 [00:00<?, ?it/s]

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in testdataloader:
    b = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**b)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

In [ ]:
model.save_pretrained('./fine-tuned-model')